In [2]:
import requests
import random as rd
import re
import time
import pandas as pd
from datetime import datetime
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains


# 상의
수집 데이터
카테고리, 아이템품번, 아이템 이름, 한정판매, 단독판매, 색상, 타겟 성별, 구매 성별, 구매 나이, 1개월 조회수, 좋아요 수, 가격(회원가), 배송정보, 평점, 후기수, Q&A 수, 1년 누적 판매량

In [3]:
# driver.close()

In [4]:
# 상의 페이지(1 page)
driver = webdriver.Chrome()
driver.get("https://www.musinsa.com/categories/item/001?d_cat_cd=001&brand=&list_kind=small&sort=pop_category&sub_sort=&page={}&display_cnt=90&group_sale=&exclusive_yn=&sale_goods=&timesale_yn=&ex_soldout=&plusDeliveryYn=&kids=&color=&price1=&price2=&shoeSizeOption=&tags=&campaign_id=&includeKeywords=&measure=".format(1))
driver.maximize_window()

In [5]:
# 제품 선택 (임의로 1의 제품을 선택하여 수집)
first_content = driver.find_element(By.CSS_SELECTOR, '#searchList > li:nth-child(1) > div.li_inner > div.article_info > p.list_info > a')
first_content.click()
# searchList > li:nth-child(9) > div.li_inner > div.article_info > p.item_title > a
# li:nth-child(9) 여기서 괄호 안에 숫자가 제품의 추천순

In [6]:
req = driver.page_source
soup = BeautifulSoup(req, 'html.parser')

#### 대분류, 중분류

In [7]:
category1 = soup.select_one('div.product-detail__sc-up77yl-0.htaeEr').get_text().split('\xa0')[0]
category1

'상의'

In [8]:
category2 = soup.select_one('div.product-detail__sc-up77yl-0.htaeEr').get_text().split('\xa0')[2]
category2

'맨투맨/스웨트셔츠'

#### 아이템 이름

In [9]:
# <span class="product_title"><em>하트 벌룬 스웻 셔츠-레드</em><span class="product_title_eng">Heart Balloon Sweat Shirt-Red</span> </span>
name = soup.select_one('h3.product-detail__sc-1klhlce-3.fjguJZ').get_text()
name

'1992 오버핏 스웨트셔츠 화이트멜란지'

#### 아이템 품번

In [10]:
def 품번():
    overlap = []
    for i in soup.select('.product-detail__sc-achptn-4.flVcwF'):
        overlap.append(i.get_text())

    if '품번' in overlap:
        index_num = overlap.index('품번')

        if overlap[index_num+1][0] == ' ':
            return overlap[index_num+1][1:]
        return overlap[index_num+1]
    else: return None

품번()

'5005601539'

### 한정판매, 무신사 단독 통합
0: False, 1: True

In [11]:
# 해당 태그가 없는 상품이 있을까봐 예외처리
def 한정_단독():
    '''
    return: limited_sales, exclusive_sale\n(한정판매 여부, 단독 여부) 튜플입니다.
    '''
    try:
        sales = soup.select_one('h4.product-detail__sc-1qpt7wc-1.dbnyPg').get_text()
        if sales == '한정판매':
            limited_sales = 1
        else:
            limited_sales = 0

        if sales == '무신사 단독':
            exclusive_sale = 1
        else:
            exclusive_sale = 0

    except:
        limited_sales = 0
        exclusive_sale = 0

    return limited_sales, exclusive_sale

한정_단독()

(0, 0)

### 타겟 성별
- 남자 = 0
- 여자 = 1
- 남/여 = 2

In [12]:
def 성별():
    overlap = []
    for i in soup.select('.product-detail__sc-achptn-4.flVcwF'):
        overlap.append(i.get_text())
    if '성별' in overlap:
        index_num = overlap.index('성별')
        # return '아아',overlap[index_num])
        if overlap[index_num-1] == '시즌':
            return overlap[index_num+2]
        else: return overlap[index_num+1]
    else: return None

def 성별2():
    if 성별() is None:
        return 성별()
    else:
        if 성별() == '남성':
            return  0
        elif 성별() == '여성':
            return  1
        else: return  2
성별2()

0

### 구매성별
- 남 = 0
- 여 = 1

In [13]:
# man graph_set_text
# 남성일 때와 여성일 때 태그가 다름
def 구매성별():
    try:
        if soup.select_one('em.product-detail__sc-wep3gk-3.kJJbYq').get_text() == '남성':
            buy_gender = 0
    except:
        try:
            if soup.select_one('em.product-detail__sc-wep3gk-3.fdjMAR').get_text() == '여성':
                buy_gender = 1
        except:   #항목 자체가 없을 때
            buy_gender = None

    return buy_gender

구매성별()

0

### 구매 나이

In [14]:
def 구매나이():
    '''
    return : (age_name_1, age_name_2) 튜플입니다.
    '''

    try:
        # 해당 엘리먼트가 있는 곳까지 스크롤
        target_element = driver.find_element(By.XPATH, '//*[@id="page_product_detail"]/div[2]/div[7]/ul/li[1]/h3')   # 구매나이 클릭
        driver.execute_script("arguments[0].scrollIntoView(true);", target_element)
        target_element.click()

        respone = driver.page_source
        soup = BeautifulSoup(respone, 'html.parser')

        age_name = soup.select('ul.bar_wrap dl:has(span.bar.hightest) dt.bar_name')

        try:
            if age_name[0] in age_name or age_name[1] in age_name:
                age_name_1 = ','.join(age_name[0]).split()
                age_name_1 = age_name_1[0]
                age_name_2 = ','.join(age_name[1]).split()
                age_name_2 = age_name_2[0]

        except:
            age_name_2 = 0
            # age_name[0]은 필히 존재 하고 age_name[1]의 유무로 인하여 오류가 발생하므로
            # 오류 발생 시 age_name[1]의 텍스트 값인 age_name_2만 '0' 으로 설정
    except:
        age_name_1 = 0
        age_name_2 = 0

    return age_name_1, age_name_2

구매나이()

('40세~', 0)

### 1개월 조회수

In [15]:
def 조회수():
    overlap = []
    for i in soup.select('.product-detail__sc-achptn-4.flVcwF'):
        overlap.append(i.get_text())

    if '조회수(1개월)' in overlap:
        index_num = overlap.index('조회수(1개월)')
        sale = overlap[index_num+1][:-4]

        if sale[-2] =='만':
            return float(sale[:-2]) * 10000
        elif sale[-2] =='천':
            return float(sale[:-2]) * 1000
        else: return float(sale)
    else: return None

def 조회수2():
    some_tag = driver.find_element(By.CSS_SELECTOR, '#page_product_detail > div.right_area.page_detail_product > div.section_graph_detail > ul > li.graph_title_viewcount > h3')
    action = ActionChains(driver)
    action.move_to_element(some_tag).perform()

    first_content = driver.find_element(By.CSS_SELECTOR, '#page_product_detail > div.right_area.page_detail_product > div.section_graph_detail > ul > li.graph_title_viewcount > h3')
    first_content.click()

    req = driver.page_source
    soup = BeautifulSoup(req, 'html.parser')

    view = soup.select_one('ul.graph_sub_title>li>h4>strong').get_text().split(' 회')[0]

    if view[-1] =='만':
        return float(view[:-1]) * 10000
    elif view[-1] =='천':
        return float(view[:-1]) * 1000
    else: return float(view.split('회')[0])

try: print(조회수2())
except: print(조회수())

65000.0


### 좋아요 수

In [16]:
def 좋아요():
    try:
        tmp_like = int(soup.select_one('span.product-detail__sc-achptn-4.flUHrZ').get_text().replace(',', ''))
    except:
        tmp_like = 0

    return tmp_like

좋아요()

1275

### 가격

In [17]:
def 가격():
    #일반
    try:
        price_element = soup.select('div.product-detail__sc-1p1ulhg-6.bIRSux span.product-detail__sc-1p1ulhg-7.hSwsZE')
        price_element = ','.join(price_element[0]).split()

        if len(price_element) == 3: # 30,000원 ~ 40,000원
            price = float(price_element[2].replace(',', '').replace('원',''))
        else: # 40,000원
            price = float(price_element[0].replace(',', '').replace('원',''))
    #클리어런스
    except:
        price = float(soup.select_one('span.product-detail__sc-1p1ulhg-7.kqpaoP').get_text()[:-1].replace(',', ''))

    return price
가격()

36900.0

### 할인율

In [18]:
def 할인율():
    if soup.select_one('div.product-detail__sc-1p1ulhg-14.gNkVLj')!=None:
        discount=soup.select_one('div.product-detail__sc-1p1ulhg-14.gNkVLj').get_text().split('%')[0]

    else :
        discount = 0

    return discount
할인율()

'50'

### 배송 일수

In [19]:
def get_date_difference(estimated_date):
    date_format = '%Y/%m/%d'  # 대문자 Y는 연도를 4자리로 표시
    target_date = datetime.strptime(estimated_date, date_format)

    # 오늘 날짜를 구합니다.
    today = datetime.now()

    # 날짜 차이를 계산합니다.
    difference = today - target_date

    return abs(difference.days)

def 배송일자():
        arrival_text = soup.select_one(".product-detail__sc-1ys1x0v-4.bPYIon")
        if arrival_text is not None:
            date_text = arrival_text.get_text().split('(')[0]
            estimated_date = '2024/' + date_text

            return  get_date_difference(estimated_date)

        else:
            # 배송 예정일이 없을 경우
            a = []
            for i in soup.select('span.product-detail__sc-1ts9zk8-4.hwqlaH'):
                a.append(i.get_text())

            try:
                if '출고 정보' in a:
                    print('출고 정보 + 2 :')
                    num = a.index('출고 정보') # 0번 # '출고 정보'가 없을 경우 에러가 뜸!
                    return  int(a[num+1].split(' ')[1].split('일')[0]) + 2
                else:
                    return  None
            except:
                 print('출고 정보에 날짜 없음(예: 13시 까지 주문 시 당일 출고)')
                 return None

배송일자()

6

### 평점

In [20]:
def 평점():
    try:
        tmp_pro_rating = float(soup.select_one('.product-detail__sc-achptn-4.iKKecn').get_text())
    except:
        tmp_pro_rating = None

    return tmp_pro_rating

평점()

4.9

### 후기 수

In [21]:
def 후기수():
    try:
        tmp_review = int(soup.select_one('#review_total').get_text().replace(',', ''))
    except:
        tmp_review = None

    return tmp_review

후기수()

44

### 구매수

In [22]:
def 누적판매():
    overlap = []
    for i in soup.select('.product-detail__sc-achptn-4.flVcwF'):
        overlap.append(i.get_text())

    if '누적판매(1년)' in overlap:
        index_num = overlap.index('누적판매(1년)')
        sale = overlap[index_num+1][:-4]
        if sale[-2] =='만':
            return float(sale[:-2]) * 10000
        elif sale[-2] =='천':
            return float(sale[:-2]) * 1000
        else: return float(sale)
    else: return None

누적판매()

2400.0

### 리뷰 크롤링1

In [23]:
# 리뷰 크롤링 함수1
def review_crawling(soup):

    # 유저명
    tmp_user_names = []

    for i in soup.select('p.review-profile__name'):
        if i.get_text() == '탈퇴회원':
            tmp_result = '탈퇴회원'

        else:
            tmp_result = i.get_text().split(' ', maxsplit=1)[1]
        tmp_user_names.append(tmp_result)

    # 유저 평점
    tmp_user_ratings = []
    search_review_rating = re.compile('\d+')

    for i in soup.select('span.review-list__rating__active'):
        tmp_result = search_review_rating.findall(str(i))[0]
        tmp_user_ratings.append(tmp_result)

    # 유저 리뷰
    tmp_user_reviews = []

    for i in soup.select('div.review-contents__text'):
        tmp_result = i.get_text()
        tmp_user_reviews.append(tmp_result)

    # 상품이름
    name = soup.select_one('h3.product-detail__sc-1klhlce-3.fjguJZ').get_text()
    tmp_names = [name] * len(tmp_user_ratings)

    # 품번
    number = soup.select_one('li.product-detail__sc-achptn-2.dEZkTF > div.product-detail__sc-achptn-6.gfoaTb > span.product-detail__sc-achptn-4.flVcwF').get_text()
    if number[0] == ' ':   # 앞 자리 공백 제거
        number = number[1:]
    tmp_numbers = [number] * len(tmp_user_ratings)

    return tmp_names, tmp_numbers, tmp_user_names, tmp_user_ratings, tmp_user_reviews

### 리뷰 크롤링2

In [24]:
# 리뷰 크롤링 함수2 (함수1 사용)
def get_tmp_df_musinsa_review_raw(driver, tag_dict):
    '''
    ex) tag_dict = {'name1':'tag1', ...}

    '''
    tmp_names = []
    tmp_numbers = []
    tmp_user_names = []
    tmp_user_ratings = []
    tmp_user_reviews = []

    # 후기 클릭-----------------------------------------------------------------------------------------------
    for key in tag_dict.keys():

        driver.find_element(By.ID, tag_dict[key]).click()
        time.sleep(1)
        # 후기 리뷰 개수
        search_review_num = re.compile('\d+[,]?\d+')
        try:

            tmp_review_count = int(search_review_num.findall(driver.find_element(By.ID, tag_dict[key]).text)[0])
        except:
            try:

                tmp_review_count = int(search_review_num.findall(driver.find_element(By.ID, tag_dict[key]).text)[0].replace(',', ''))
            except:
                search_review_num = re.compile('\d+')
                tmp_review_count = int(search_review_num.findall(driver.find_element(By.ID, tag_dict[key]).text)[0])

        if tmp_review_count == 0:
            print(f'{key} 없음')
            # 다음 리뷰 크롤링으로 이동

        else:
            # 리뷰 1페이지 크롤링
            req = driver.page_source
            soup = BeautifulSoup(req, 'html.parser')

            tmp_names1, tmp_numbers1, tmp_user_names1, tmp_user_ratings1, tmp_user_reviews1 = review_crawling(soup)
            # print(f'{key} 1페이지 유저 이름: {tmp_user_names1}')

            tmp_names = tmp_names + tmp_names1
            tmp_numbers = tmp_numbers + tmp_numbers1
            tmp_user_names = tmp_user_names + tmp_user_names1
            tmp_user_ratings = tmp_user_ratings + tmp_user_ratings1
            tmp_user_reviews = tmp_user_reviews + tmp_user_reviews1

            # 리뷰가 10개 초과여서 다음 페이지 크롤링이 필요한 경우
            if tmp_review_count > 10:
                # 후기 다음 페이지 클릭
                # 다른 요소로 스크롤하여 클릭하려는 요소가 보이도록 만들기
                # from selenium.webdriver.common.keys import Keys

                try:
                    target_element = driver.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[4]')   #2페이지
                    driver.execute_script("arguments[0].scrollIntoView(true);", target_element)
                    target_element.click()

                except ElementClickInterceptedException as e:
                    print("2페이지로 넘어갈 수 없습니다.")

                # time.sleep(1)
                # driver.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[4]').click()
                time.sleep(1)

                req = driver.page_source
                soup = BeautifulSoup(req, 'html.parser')

                tmp_names2, tmp_numbers2, tmp_user_names2, tmp_user_ratings2, tmp_user_reviews2 = review_crawling(soup)
                # print(f'{key} 2페이지 유저 이름: {tmp_user_names2}')

                tmp_names = tmp_names + tmp_names2[:5]
                tmp_numbers = tmp_numbers + tmp_numbers2[:5]
                tmp_user_names = tmp_user_names + tmp_user_names2[:5]
                tmp_user_ratings = tmp_user_ratings + tmp_user_ratings2[:5]
                tmp_user_reviews = tmp_user_reviews + tmp_user_reviews2[:5]
                # print(f'{key} 수집 후 유저 이름: {tmp_user_names}')


            list_len = 15 if tmp_review_count >= 15 else tmp_review_count
            print(f'{key} 수집: {list_len} 개')

    return tmp_names, tmp_numbers, tmp_user_names, tmp_user_ratings, tmp_user_reviews

In [25]:
driver.close()



---



## 자동화

In [41]:
# 실제로는 100페이지 정도를 추출해서 각 페이지 당 10개씩 추출하는 등으로 진행
page = sorted(rd.sample(range(28, 30), 2))
page, len(page)

([28, 29], 2)

In [42]:
# 페이지 내에 상품 랜덤 추출
import random as rd

random_product = rd.sample(range(1, 91), 90)
random_product = sorted(random_product)
len(random_product)

90

In [43]:
import time
from selenium.webdriver.chrome.options import Options
from tqdm import tqdm_notebook


options = Options()
options.headless = True # 헤드리스 모드로 지정해 크롬을 GUI 없이 수행
# 상의 페이지(1 page)

major_category = [] # 대분류
middle_category = [] # 중분류
names = [] # 제품명
numbers = [] # 품번
limit = [] # 한정판매
exclusive = [] # 단독판매
target_gen = [] # 타겟성별
buy_gen = [] # 구매 성별
buy_age = [] # 구매 나이
month_view = [] # 1개월 조회수
like_cnt = [] # 좋아요 수
pro_price = [] # 판매 가격
discount_rate=[] # 할인률
delivery_date = [] # 배송일자
pro_rating = [] # 평점
review = [] # 리뷰 수
buy = [] # 1년 간 누적 판매

df_musinsa_review_raw = pd.DataFrame(columns=['product_names', 'product_ids', 'user_names', 'user_ratings', 'user_reviews'])   #리뷰 수집df
tag_dict = {'스타일 후기': 'estimate_style', '상품 후기': 'estimate_photo', '일반 후기': 'estimate_goods'}

driver = webdriver.Chrome()
for i in tqdm_notebook(page):
    # 페이지 열기
    driver.get("https://www.musinsa.com/categories/item/003?d_cat_cd=003&brand=&list_kind=small&sort=pop_category&sub_sort=&page={}&display_cnt=90&group_sale=&exclusive_yn=&sale_goods=&timesale_yn=&ex_soldout=&plusDeliveryYn=&kids=&color=&price1=&price2=&shoeSizeOption=&tags=&campaign_id=&includeKeywords=&measure=".format(i))
    try:
        driver.maximize_window()
    except:
        pass

    for j in random_product:
        print(f'{i}페이지 {j}번 리뷰 크롤링 시작-------------------')

        try:
        # 제품 클릭하기 여기서 화면 크기때문에 클릭이 안될 수 있음. if문으로 스크롤 넣어주기
            first_content = driver.find_element(By.CSS_SELECTOR, '#searchList > li:nth-child({}) > div.li_inner > div.article_info > p.list_info > a'.format(j))
            first_content.click()
            time.sleep(1.5)

        except:
            driver.get("https://www.musinsa.com/categories/item/003?d_cat_cd=003&brand=&list_kind=small&sort=pop_category&sub_sort=&page={}&display_cnt=90&group_sale=&exclusive_yn=&sale_goods=&timesale_yn=&ex_soldout=&plusDeliveryYn=&kids=&color=&price1=&price2=&shoeSizeOption=&tags=&campaign_id=&includeKeywords=&measure=".format(i))
            first_content = driver.find_element(By.CSS_SELECTOR, '#searchList > li:nth-child({}) > div.li_inner > div.article_info > p.list_info > a'.format(j+1))
            first_content.click()
            time.sleep(1.5)

        # 화면 html 코드 가져오기
        req = driver.page_source
        soup = BeautifulSoup(req, 'html.parser')

        # 대분류와 중분류
        try:
            category1 = soup.select_one('div.product-detail__sc-up77yl-0.htaeEr').get_text().split('\xa0')[0]
            major_category.append(category1)

            category2 = soup.select_one('div.product-detail__sc-up77yl-0.htaeEr').get_text().split('\xa0')[2]
            middle_category.append(category2)
        except:
            time.sleep(1.5)
            category1 = soup.select_one('div.product-detail__sc-up77yl-0.htaeEr').get_text().split('\xa0')[0]
            major_category.append(category1)

            category2 = soup.select_one('div.product-detail__sc-up77yl-0.htaeEr').get_text().split('\xa0')[2]
            middle_category.append(category2)

        # 이름
        name = soup.select_one('h3.product-detail__sc-1klhlce-3.fjguJZ').get_text()
        names.append(name)

        # 품번
        numbers.append(품번())

        # 한정판매
        limit.append(한정_단독()[0])

        # 단독판매
        exclusive.append(한정_단독()[1])

        # 타겟 젠더
        target_gen.append(성별2())

        # 구매 성별
        buy_gen.append(구매성별())

        # 구매 나이
        buy_age.append(구매나이())

        # 1개월 조회수
        try:
            month_view.append(조회수2())
        except:
            month_view.append(조회수())

        # 좋아요 수
        like_cnt.append(좋아요())

        # 가격(가격)
        pro_price.append(가격())

        # 할인율
        discount_rate.append(할인율())

        # 배송일자
        delivery_date.append(배송일자())

        # 평점
        pro_rating.append(평점())

        # 후기수
        review.append(후기수())

        # 구매수
        buy.append(누적판매())

        #리뷰 크롤링
        product_names, product_ids, user_names, user_ratings, user_reviews = get_tmp_df_musinsa_review_raw(driver, tag_dict)

        tmp_df_musinsa_review_raw = pd.DataFrame({'product_names': product_names, 'product_ids': product_ids, 'user_names': user_names, 'user_ratings': user_ratings, 'user_reviews': user_reviews})

        df_musinsa_review_raw = pd.concat([df_musinsa_review_raw, tmp_df_musinsa_review_raw], axis=0)
        print(f'Total DF rows: {df_musinsa_review_raw.count()[1]}')
        driver.back()
        time.sleep(1)

C:\Users\user\AppData\Local\Temp\ipykernel_19972\4091787821.py:32: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(page):


  0%|          | 0/2 [00:00<?, ?it/s]

28페이지 1번 리뷰 크롤링 시작-------------------
스타일 후기 없음
상품 후기 없음
일반 후기 없음
Total DF rows: 0
28페이지 2번 리뷰 크롤링 시작-------------------
출고 정보 + 2 :
스타일 후기 없음
상품 후기 없음
일반 후기 없음
Total DF rows: 0
28페이지 3번 리뷰 크롤링 시작-------------------
스타일 후기 없음
상품 후기 없음
일반 후기 없음
Total DF rows: 0
28페이지 4번 리뷰 크롤링 시작-------------------
스타일 후기 없음
상품 후기 없음
일반 후기 없음
Total DF rows: 0
28페이지 5번 리뷰 크롤링 시작-------------------
출고 정보 + 2 :
스타일 후기 없음
상품 후기 없음
일반 후기 없음
Total DF rows: 0
28페이지 6번 리뷰 크롤링 시작-------------------
출고 정보 + 2 :
스타일 후기 없음
상품 후기 없음
일반 후기 없음
Total DF rows: 0
28페이지 7번 리뷰 크롤링 시작-------------------
스타일 후기 없음
상품 후기 없음
일반 후기 없음
Total DF rows: 0
28페이지 8번 리뷰 크롤링 시작-------------------
출고 정보 + 2 :
스타일 후기 없음
상품 후기 없음
일반 후기 없음
Total DF rows: 0
28페이지 9번 리뷰 크롤링 시작-------------------
스타일 후기 없음
상품 후기 없음
일반 후기 없음
Total DF rows: 0
28페이지 10번 리뷰 크롤링 시작-------------------
스타일 후기 수집: 15 개
상품 후기 수집: 15 개
일반 후기 수집: 15 개
Total DF rows: 45
28페이지 11번 리뷰 크롤링 시작-------------------
출고 정보 + 2 :
스타일 후기 없음
상품 후기 수집: 1 개
일반 후기 수집: 1 개
Total DF rows

In [44]:
len(major_category),len(middle_category),len(names), len(numbers), len(limit), len(exclusive), len(target_gen), len(buy_gen), len(month_view), len(like_cnt), len(pro_price),len(discount_rate) ,len(delivery_date), len(pro_rating), len(review), len(buy)

(180,
 180,
 180,
 180,
 180,
 180,
 180,
 180,
 180,
 180,
 180,
 180,
 180,
 180,
 180,
 180)

In [45]:
import pandas as pd

df = pd.DataFrame({'major_category':major_category,'middle_category': middle_category,'name': names, 'number': numbers, 'limit': limit, 'exclusive': exclusive, 'target_gender': target_gen,
              'buy_gender': buy_gen, 'buy_age' : buy_age, 'view': month_view, 'like': like_cnt, 'price': pro_price, 'discount_rate':discount_rate, 'delivery_date': delivery_date,
              'rating': pro_rating, 'review': review, 'buy':buy})

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   major_category   180 non-null    object 
 1   middle_category  180 non-null    object 
 2   name             180 non-null    object 
 3   number           180 non-null    object 
 4   limit            180 non-null    int64  
 5   exclusive        180 non-null    int64  
 6   target_gender    180 non-null    int64  
 7   buy_gender       122 non-null    float64
 8   buy_age          180 non-null    object 
 9   view             141 non-null    float64
 10  like             180 non-null    int64  
 11  price            180 non-null    float64
 12  discount_rate    180 non-null    object 
 13  delivery_date    180 non-null    int64  
 14  rating           140 non-null    float64
 15  review           180 non-null    int64  
 16  buy              121 non-null    float64
dtypes: float64(5), i

In [47]:
df.isnull().sum()

major_category      0
middle_category     0
name                0
number              0
limit               0
exclusive           0
target_gender       0
buy_gender         58
buy_age             0
view               39
like                0
price               0
discount_rate       0
delivery_date       0
rating             40
review              0
buy                59
dtype: int64

In [48]:
df.drop_duplicates()

,major_category,middle_category,name,number,limit,exclusive,target_gender,buy_gender,buy_age,view,like,price,discount_rate,delivery_date,rating,review,buy
0,바지,트레이닝/조거 팬츠,Color block jogger pant (khaki grey),22FWCJPT01KK,0,0,0,NaN,"(19~23세, 0)",200.0,205,98000.0,0,5,NaN,0,NaN
1,바지,트레이닝/조거 팬츠,VTG 코튼 니트 조거팬츠 멜란지 그레이,OESW1JME,0,0,0,NaN,"(0, 0)",NaN,4,128000.0,0,5,NaN,0,NaN
2,스포츠/용품,하의,요가웨어B 숏팬츠 01-1,12834740,0,0,1,NaN,"(0, 0)",NaN,8,12700.0,0,5,NaN,0,NaN
3,바지,기타 바지,SR_Commuter Straight Casual Pants_2color,m1037_1144243,0,0,1,NaN,"(0, 0)",NaN,2,164000.0,0,10,NaN,0,NaN
4,바지,트레이닝/조거 팬츠,LEAF SWEATPANTS,20230610,0,0,0,NaN,"(0, 0)",NaN,9,129000.0,0,5,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,바지,숏 팬츠,Faux Leather Shorts - White,1000,0,0,0,NaN,"(0, 0)",NaN,8,130000.0,0,6,NaN,0,NaN
176,바지,데님 팬츠,데이 와이드 밴딩 데님 팬츠 (LIGHT BLUE),T3S-DWBDPT21,0,0,0,0.0,"(24~28세, 29~33세)",5000.0,1849,39000.0,20,5,4.9,228,1200.0
177,바지,트레이닝/조거 팬츠,L.U.P BY LIPUNDERPOINT NEVER DIE SWEAT PANTS_GRAY,LP18SLPT001GR,0,0,0,NaN,"(0, 0)",NaN,1,71200.0,20,5,NaN,0,NaN
178,바지,데님 팬츠,그레이시 버튼 다운 스트레이트 진,ND1BPT07GY,0,0,1,NaN,"(0, 0)",NaN,7,109000.0,0,5,NaN,0,NaN


In [49]:
df_musinsa_review_raw

,product_names,product_ids,user_names,user_ratings,user_reviews
0,데일리 카고 쇼츠 (KHAKI GRAY),atm20ub002kg,추멘아들,100,카키그레이라도 조금 더 밟은편인 거 같아요 시원하고 편해요
1,데일리 카고 쇼츠 (KHAKI GRAY),atm20ub002kg,wittyyy,100,생각보다 더 짧긴 하지만 놀러갈 때 편하게 입기 좋을 것 같아용
2,데일리 카고 쇼츠 (KHAKI GRAY),atm20ub002kg,뉴비_c18ac68a29da,100,바지가 편하고 길이도 적당함.클라이밍이 다리를 많이 올리고 찢는동작이 믾은데 불편하...
3,데일리 카고 쇼츠 (KHAKI GRAY),atm20ub002kg,Didoro,100,"아음,저가 살때 76이였고 75까지 빠졌는데 그냥 정사이즈 입니다 저는 골근량이 3..."
4,데일리 카고 쇼츠 (KHAKI GRAY),atm20ub002kg,돈없는남성,100,"여름에 입기 좋고, 색감이 생각했던거보다 이뻐요!"
...,...,...,...,...,...
40,데이 와이드 밴딩 데님 팬츠 (LIGHT BLUE),T3S-DWBDPT21,connor9,100,잘 입고 있습니다. 흰색상의랑 잘 어울려요. 많이 파세용.
41,데이 와이드 밴딩 데님 팬츠 (LIGHT BLUE),T3S-DWBDPT21,후후포,100,색감도 적당히 좋고 넉넉하니 편하게 입기 좋습니다
42,데이 와이드 밴딩 데님 팬츠 (LIGHT BLUE),T3S-DWBDPT21,뉴비_2d1393492d99,100,밴드라서 너무 편합니다 중요한건 핏인데 배꼽까지 올려입으면 새미 와이드 느낌이고 대...
43,데이 와이드 밴딩 데님 팬츠 (LIGHT BLUE),T3S-DWBDPT21,코부다,100,쥑입니다 너무좋아요 배송빠르고 가격좋고 디자인이쁘고색감도 미쳤고 핏도 너무좋음


In [51]:
df.to_csv('../ML 팀프로젝트/csv/[이병찬]musinsa_하의_240119_2.csv')

In [50]:
df_musinsa_review_raw.to_csv('../ML 팀프로젝트/csv/[이병찬]review_하의_240119_2.csv')

In [67]:
df1 = pd.read_csv('./csv/[이병찬]musinsa_하의_240119_1.csv')

In [68]:
df1

,Unnamed: 0,major_category,middle_category,name,number,limit,exclusive,target_gender,buy_gender,buy_age,view,like,price,discount_rate,delivery_date,rating,review,buy
0,0,바지,숏 팬츠,라인 와이드 반바지-머스타드,21063001,0,0,0,NaN,"(0, 0)",NaN,36,29500.0,50,6.0,5.0,4,NaN
1,1,바지,숏 팬츠,버뮤다 팬츠 (차콜),P0000FWI,0,0,1,NaN,"(0, 0)",NaN,39,72000.0,0,6.0,NaN,0,NaN
2,2,스포츠/용품,하의,[W]JWO 트레이닝 셋업 팬츠_BK,22FWGWPT04_BK,0,0,1,NaN,"(0, 0)",NaN,2,75600.0,60,5.0,NaN,0,NaN
3,3,바지,데님 팬츠,하프 데님 쇼츠 - 8% 멜란지 #1,P00000HY000A,0,0,0,NaN,"(0, 0)",NaN,3,164000.0,0,5.0,NaN,0,NaN
4,4,바지,코튼 팬츠,MAD CORDUROY PANTS_BK,MD21FWCP_BK,0,0,0,NaN,"(0, 0)",NaN,10,88000.0,10,5.0,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,271,바지,트레이닝/조거 팬츠,VTG 코튼 니트 조거팬츠 멜란지 그레이,OESW1JME,0,0,0,NaN,"(0, 0)",NaN,4,128000.0,0,5.0,NaN,0,NaN
272,272,스포츠/용품,하의,요가웨어B 숏팬츠 01-1,12834740,0,0,1,NaN,"(0, 0)",NaN,8,12700.0,0,5.0,NaN,0,NaN
273,273,바지,기타 바지,SR_Commuter Straight Casual Pants_2color,m1037_1144243,0,0,1,NaN,"(0, 0)",NaN,2,164000.0,0,10.0,NaN,0,NaN
274,274,바지,트레이닝/조거 팬츠,LEAF SWEATPANTS,20230610,0,0,0,NaN,"(0, 0)",NaN,9,129000.0,0,5.0,NaN,0,NaN


In [69]:
df2 = pd.read_csv('./csv/[이병찬]musinsa_하의_240119_2.csv')

In [70]:
df2

,Unnamed: 0,major_category,middle_category,name,number,limit,exclusive,target_gender,buy_gender,buy_age,view,like,price,discount_rate,delivery_date,rating,review,buy
0,0,바지,트레이닝/조거 팬츠,Color block jogger pant (khaki grey),22FWCJPT01KK,0,0,0,NaN,"('19~23세', 0)",200.0,205,98000.0,0,5,NaN,0,NaN
1,1,바지,트레이닝/조거 팬츠,VTG 코튼 니트 조거팬츠 멜란지 그레이,OESW1JME,0,0,0,NaN,"(0, 0)",NaN,4,128000.0,0,5,NaN,0,NaN
2,2,스포츠/용품,하의,요가웨어B 숏팬츠 01-1,12834740,0,0,1,NaN,"(0, 0)",NaN,8,12700.0,0,5,NaN,0,NaN
3,3,바지,기타 바지,SR_Commuter Straight Casual Pants_2color,m1037_1144243,0,0,1,NaN,"(0, 0)",NaN,2,164000.0,0,10,NaN,0,NaN
4,4,바지,트레이닝/조거 팬츠,LEAF SWEATPANTS,20230610,0,0,0,NaN,"(0, 0)",NaN,9,129000.0,0,5,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,175,바지,숏 팬츠,Faux Leather Shorts - White,1000,0,0,0,NaN,"(0, 0)",NaN,8,130000.0,0,6,NaN,0,NaN
176,176,바지,데님 팬츠,데이 와이드 밴딩 데님 팬츠 (LIGHT BLUE),T3S-DWBDPT21,0,0,0,0.0,"('24~28세', '29~33세')",5000.0,1849,39000.0,20,5,4.9,228,1200.0
177,177,바지,트레이닝/조거 팬츠,L.U.P BY LIPUNDERPOINT NEVER DIE SWEAT PANTS_GRAY,LP18SLPT001GR,0,0,0,NaN,"(0, 0)",NaN,1,71200.0,20,5,NaN,0,NaN
178,178,바지,데님 팬츠,그레이시 버튼 다운 스트레이트 진,ND1BPT07GY,0,0,1,NaN,"(0, 0)",NaN,7,109000.0,0,5,NaN,0,NaN


In [71]:
df_review = pd.concat([df1, df2])

In [73]:
df_review.drop_duplicates()

,Unnamed: 0,major_category,middle_category,name,number,limit,exclusive,target_gender,buy_gender,buy_age,view,like,price,discount_rate,delivery_date,rating,review,buy
0,0,바지,숏 팬츠,라인 와이드 반바지-머스타드,21063001,0,0,0,NaN,"(0, 0)",NaN,36,29500.0,50,6.0,5.0,4,NaN
1,1,바지,숏 팬츠,버뮤다 팬츠 (차콜),P0000FWI,0,0,1,NaN,"(0, 0)",NaN,39,72000.0,0,6.0,NaN,0,NaN
2,2,스포츠/용품,하의,[W]JWO 트레이닝 셋업 팬츠_BK,22FWGWPT04_BK,0,0,1,NaN,"(0, 0)",NaN,2,75600.0,60,5.0,NaN,0,NaN
3,3,바지,데님 팬츠,하프 데님 쇼츠 - 8% 멜란지 #1,P00000HY000A,0,0,0,NaN,"(0, 0)",NaN,3,164000.0,0,5.0,NaN,0,NaN
4,4,바지,코튼 팬츠,MAD CORDUROY PANTS_BK,MD21FWCP_BK,0,0,0,NaN,"(0, 0)",NaN,10,88000.0,10,5.0,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,175,바지,숏 팬츠,Faux Leather Shorts - White,1000,0,0,0,NaN,"(0, 0)",NaN,8,130000.0,0,6.0,NaN,0,NaN
176,176,바지,데님 팬츠,데이 와이드 밴딩 데님 팬츠 (LIGHT BLUE),T3S-DWBDPT21,0,0,0,0.0,"('24~28세', '29~33세')",5000.0,1849,39000.0,20,5.0,4.9,228,1200.0
177,177,바지,트레이닝/조거 팬츠,L.U.P BY LIPUNDERPOINT NEVER DIE SWEAT PANTS_GRAY,LP18SLPT001GR,0,0,0,NaN,"(0, 0)",NaN,1,71200.0,20,5.0,NaN,0,NaN
178,178,바지,데님 팬츠,그레이시 버튼 다운 스트레이트 진,ND1BPT07GY,0,0,1,NaN,"(0, 0)",NaN,7,109000.0,0,5.0,NaN,0,NaN


In [74]:
df_review.to_csv('./csv/[이병찬]musinsa_하의_240119.csv')